In [754]:
import psycopg2
from psycopg2.extensions import AsIs
import psycopg2
import pandas as pd
import numpy as np

In [755]:
%run -i ../sql_helpers.py
display(db_credentials)

{'dbname': 'gta',
 'port': 5432,
 'user': 'gta_p8',
 'password': 'r7sdkfdq',
 'host': 'ikgpgis.ethz.ch'}

In [756]:
kirchen_zuerich = pd.read_csv("csv/kirchen-zurich.csv", encoding = "utf-8", sep="	")
restaurant_zurich = pd.read_csv("csv/restaurant-zurich.csv", encoding = "utf-8", sep="	")

In [757]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [758]:
sql_string_with_placeholders = "INSERT INTO church (address_id, church_name) VALUES (%s, %s)"
cur.mogrify(sql_string_with_placeholders,(100, "kirche"))

b"INSERT INTO church (address_id, church_name) VALUES (100, 'kirche')"

In [759]:
conn.commit()
conn.close()

Kirchen

In [760]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [761]:
city_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #city_table
    sql_fill_city = "INSERT INTO gta_p8.city (city_postcode, city_name) SELECT '"+city_postcode+"', '"+city_name+"' WHERE NOT EXISTS (select * from gta_p8.city where city_postcode='"+city_postcode+"');"

    cur.execute(sql_fill_city, (city_postcode, city_name))

    #address_table_preparation
    cur.execute("SELECT city_id FROM gta_p8.city WHERE city_postcode='"+city_postcode+"';")
    result = cur.fetchone()
    city_id_array = np.append(city_id_array, result[0])





In [762]:
conn.commit()
conn.close()

In [763]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [764]:
address_id_array = np.empty(0)

for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>

    #EXECUTE sql_commands:

    #address_table
    city_id_fk = str(int(city_id_array[idx]))

    sql_fill_address = "INSERT INTO gta_p8.address (address_street, address_number, city_id) SELECT '"+address_street+"', '"+address_number+"', '"+city_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.address where address_street='"+address_street+"' AND address_number='"+address_number+"');"
  
    #cur.execute(sql_fill_address, (address_street, address_number, city_id_fk))

    #kirchen_table_preparation
    cur.execute("SELECT address_id FROM gta_p8.address WHERE address_street='"+address_street+"' AND address_number='"+address_number+"';")
    result = cur.fetchone()
    address_id_array = np.append(address_id_array, result[0])

print(address_id_array)

[ 940.  941.  941.  942.  942.  943.  941.  941.  944.  941.  941.  941.
  945.  941.  946.  947.  948.  949.  950.  951.  952.  953.  954.  955.
  956.  957.  958.  959.  960.  961.  962.  963.  964.  965.  966.  967.
  968.  969.  941.  970.  971.  972.  973.  974.  975.  976.  977.  978.
  979.  980.  981.  982.  983.  984.  985.  941.  941.  986.  941.  987.
  988.  989.  990.  991.  992.  941.  993.  994.  995.  941.  996.  997.
  998.  999. 1000. 1001. 1002. 1003. 1004. 1005. 1006. 1007. 1008. 1009.
 1010. 1011. 1012. 1013. 1014. 1015. 1016. 1017.  941. 1018. 1019. 1020.
 1021. 1022. 1023.  941. 1024. 1025. 1026. 1027. 1028. 1029.  941. 1030.
 1031. 1032.  941. 1033. 1034. 1035.  941.  941.  941. 1036.  941.  941.]


In [765]:
conn.commit()
conn.close()

In [766]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

In [767]:
for idx, row in kirchen_zuerich.iterrows():
    #prepare input values
    church_name = row['name'] #<class 'str'> x
    if(type(church_name) == float): #nameless churches get skipped
        continue

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = int(city_postcode)
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'

    lon = str(row["@lon"]) #<class 'float'>

    lat = str(row["@lat"]) #<class 'float'>

    #EXECUTE sql_commands:
    address_id_fk = str(int(address_id_array[idx]))

    sql_fill_church = "INSERT INTO gta_p8.church (church_name, address_id) SELECT '"+church_name+"', '"+address_id_fk+"' WHERE NOT EXISTS (select * from gta_p8.church where church_name='"+church_name+"');"

    cur.execute(sql_fill_church, (church_name, address_id_fk))

    geom_sql = "UPDATE gta_p8.church SET geometry = ST_SetSRID(ST_MakePoint("+lon+","+lat+"),4326);"

    cur.execute(geom_sql)



In [768]:
conn.commit()
conn.close()

## Restaurants

In [769]:
for idx, row in restaurant_zurich.iterrows():
    #prepare input values
    restaurant_name = row['name'] #<class 'str'>

    restaurant_website = row['website'] #<class 'str'>
    if(type(restaurant_website) == float):
        restaurant_website = 'keine Website vorhanden!'

    restaurant_type = row['cuisine']
    if(type(restaurant_type) == float):
        restaurant_type = 'keine Kategorie!'

    address_street = row['addr:street'] #<class 'str'>
    if(type(address_street) == float):
        address_street = 'keine Adresse vorhanden!'

    address_number = row['addr:housenumber'] #<class 'str'>
    if(type(address_number) == float):
        address_number = '-'

    city_postcode = row['addr:postcode'] #<class 'str'>
    if pd.isna(city_postcode) == True:
        city_postcode = '-'
    else:
        city_postcode = str(city_postcode)

    city_name = row['addr:city'] #<class 'str'>
    if(type(city_name) == float):
        city_name = '-'   

    lon = row["@lon"] #<class 'float'>

    lat = row["@lat"] #<class 'float'>
    